In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time
import sys
import pymongo
from pymongo import MongoClient, IndexModel, ASCENDING, DESCENDING
import subprocess as sp

batch_folder = #Sostituisci con il percorso della cartella Batch files (inizializzazione sharded DB) che hai scaricato dal drive


# 1) Inizializzo gli shards ed il config server
### Ciò avviene lanciando il file .bat preparato all'occorrenza

In [2]:
#Attenzione, questo aprirà 12 terminali!
filename1 = "1) launch_shards&configsrvr.bat"

p1 = sp.Popen(batch_folder+'/'+filename1)
stout, sterr = p1.communicate()
time.sleep(180)

# 2) Inserisco le istruzioni per definire i replica sets
## (che sarebbero gli shards ed il config server).
### Questa parte va fatta solo al primo avvio

In [ ]:

shard1 = MongoClient('localhost', 27118)
shard2 = MongoClient('localhost', 27218)
shard3 = MongoClient('localhost', 27318)
configsrvr = MongoClient('localhost', 27031)
time.sleep(5)
#Inizializzo i replica sets che formeranno gli shards ed il config server (necessario solo al primo avvio o dopo un reset)

config0 = {'_id': 'configsvr', 'members': [
{'_id': 0, 'host': 'localhost:27031'},
{'_id': 1, 'host': 'localhost:27032'},
{'_id': 2, 'host': 'localhost:27033'}]}
configsrvr.admin.command("replSetInitiate", config0)


config1 = {'_id': 'kiva_s1', 'members': [
{'_id': 0, 'host': 'localhost:27118'},
{'_id': 1, 'host': 'localhost:27119'},
{'_id': 2, 'host': 'localhost:27120'}]}
shard1.admin.command("replSetInitiate", config1)


config2 = {'_id': 'kiva_s2', 'members': [
{'_id': 0, 'host': 'localhost:27218'},
{'_id': 1, 'host': 'localhost:27219'},
{'_id': 2, 'host': 'localhost:27220'}]}
shard2.admin.command("replSetInitiate", config2)


config3 = {'_id': 'kiva_s3', 'members': [
{'_id': 0, 'host': 'localhost:27318'},
{'_id': 1, 'host': 'localhost:27319'},
{'_id': 2, 'host': 'localhost:27320'}]}
shard3.admin.command("replSetInitiate", config3)

# 3) Avvio il router mongos, con cui comunicherò per inserire i dati ed effettuare le queries

In [2]:
filename2 = '2) launch_mongos.bat'
p2 = sp.run(batch_folder+'/'+filename2)
sleep(10)
client = MongoClient(host='localhost', port=27018)
db = client.kiva_DB 

# 4) Creazione indice e sharding di db e collection (da fare ASSOLUTAMENTE dopo aver fatto la parte da terminale, e comunque solo la prima volta.)

In [ ]:
db.loans.create_index([('id', ASCENDING)])
client.admin.command({'enableSharding':'kiva_DB'})
client.admin.command('shardCollection', 'kiva_DB.loans', key={'id': 1})